In [61]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

def generate_balanced_patient_data(num_patients):
    np.random.seed(0)
    data = {
        'patient_id': np.arange(1, num_patients + 1),
        'age': np.random.randint(20, 91, num_patients),
        'weight': np.random.normal(75, 15, num_patients).astype(int),
        'medical_conditions': np.random.choice(['Hypertension', 'Diabetes', 'Anemia', 'None'], num_patients, p=[0.25, 0.25, 0.25, 0.25]),
        'heartrate': np.random.normal(80, 10, num_patients).astype(int),
        'spo2': np.random.normal(95, 2, num_patients).astype(int),
        'blood_pressure_systolic': np.random.normal(120, 15, num_patients).astype(int),
        'ankle_swelling': np.random.randint(1, 6, num_patients),
        'breathlessness': np.random.randint(1, 6, num_patients)              
    }
    patient_df = pd.DataFrame(data)
    patient_df['weight'] = patient_df['weight'].clip(50, 120)
    patient_df['heartrate'] = patient_df['heartrate'].clip(60, 100)
    patient_df['spo2'] = patient_df['spo2'].clip(90, 100)
    patient_df['blood_pressure_systolic'] = patient_df['blood_pressure_systolic'].clip(90, 140)
    return patient_df

def generate_weekly_symptoms_data(patient_data, num_weeks):
    records = []
    for _, row in patient_data.iterrows():
        patient_id = row['patient_id']
        start_date = datetime.now() - timedelta(weeks=num_weeks)
        for week in range(num_weeks):
            current_date = start_date + timedelta(weeks=week)
            records.append({
                'patient_id': patient_id,
                'timestamp': current_date,
                'ankle_swelling': np.random.randint(1, 6),
                'breathlessness': np.random.randint(1, 6)
            })
            

    weekly_symptoms_df = pd.DataFrame(records)
    weekly_symptoms_df['average_symptoms'] = weekly_symptoms_df[['ankle_swelling', 'breathlessness']].mean(axis=1)
    weekly_symptoms_df['average_diff'] = weekly_symptoms_df.groupby('patient_id')['average_symptoms'].diff().fillna(0)
    
    # Determine the progress (worsening, improving, stable)
    def determine_progress(diff):
        if diff > 0:
            return 'Worsening'
        elif diff < 0:
            return 'Improving'
        else:
            return 'Stable'
    
    weekly_symptoms_df['progress'] = weekly_symptoms_df['average_diff'].apply(determine_progress)
    
    return weekly_symptoms_df

# Generate datasets
patient_data = generate_balanced_patient_data(5000)
weekly_symptoms_data = generate_weekly_symptoms_data(patient_data, 104)

# Step 1: Add a drug column to patient_data
drugs = ['Drug_A', 'Drug_B', 'Drug_C', 'Drug_D']
patient_data['drug'] = np.random.choice(drugs, size=len(patient_data))

# Step 2: Get the most recent 'average_diff' and 'progress' for each patient
latest_weekly_data = weekly_symptoms_data.sort_values(by='timestamp').groupby('patient_id').tail(1)[['patient_id', 'average_diff', 'progress']]

# Step 3: Merge the latest 'average_diff' and 'progress' with patient_data
combined_data = pd.merge(patient_data, latest_weekly_data, on='patient_id', how='left')

# Display the combined DataFrame with relevant columns

combined_data.head()

patient_id  age  weight medical_conditions  heartrate  spo2  \
0           1   64      96               None         69    97   
1           2   67      66           Diabetes         90    95   
2           3   84      98             Anemia         76    97   
3           4   87      68             Anemia         99    95   
4           5   87      67               None         83    93   

   blood_pressure_systolic  ankle_swelling  breathlessness    drug  \
0                       91               1               5  Drug_C   
1                      120               2               5  Drug_B   
2                      108               1               4  Drug_A   
3                      116               3               2  Drug_C   
4                      125               5               3  Drug_D   

   average_diff   progress  
0           0.5  Worsening  
1          -2.0  Improving  
2           0.0     Stable  
3           1.0  Worsening  
4           3.5  Worsening

In [62]:
# Define the ranges for each category
heartrate_bins = [0, 60, 70, 80, 90, 100, float('inf')]
spo2_bins = [0, 90, 92, 94, 96, 98, 100]
bp_systolic_bins = [0, 90, 100, 110, 120, 130, float('inf')]
#bp_diastolic_bins = [0, 60, 70, 80, 90, float('inf')]

# Create categorical variables
combined_data['heartrate_category'] = pd.cut(combined_data['heartrate'], bins=heartrate_bins, labels=[1, 2, 3, 4, 5, 6])
combined_data['spo2_category'] = pd.cut(combined_data['spo2'], bins=spo2_bins, labels=[1, 2, 3, 4, 5, 6])
combined_data['blood_pressure_systolic_category'] = pd.cut(combined_data['blood_pressure_systolic'], bins=bp_systolic_bins, labels=[1, 2, 3, 4, 5, 6])

# Display the updated DataFrame
combined_data.head()

patient_id  age  weight medical_conditions  heartrate  spo2  \
0           1   64      96               None         69    97   
1           2   67      66           Diabetes         90    95   
2           3   84      98             Anemia         76    97   
3           4   87      68             Anemia         99    95   
4           5   87      67               None         83    93   

   blood_pressure_systolic  ankle_swelling  breathlessness    drug  \
0                       91               1               5  Drug_C   
1                      120               2               5  Drug_B   
2                      108               1               4  Drug_A   
3                      116               3               2  Drug_C   
4                      125               5               3  Drug_D   

   average_diff   progress heartrate_category spo2_category  \
0           0.5  Worsening                  2             5   
1          -2.0  Improving                  4             4   
2           0.0     Stable                  3             5   
3           1.0  Worsening                  5             4   
4           3.5  Worsening                  4             3   

  blood_pressure_systolic_category  
0                                2  
1                                4  
2                                3  
3                                4  
4                                5

In [63]:
def generate_weekly_symptoms_data(patient_data, num_weeks):
    records = []
    for _, row in patient_data.iterrows():
        patient_id = row['patient_id']
        start_date = datetime.now() - timedelta(weeks=num_weeks)
        for week in range(num_weeks):
            current_date = start_date + timedelta(weeks=week)
            records.append({
                'patient_id': patient_id,
                'timestamp': current_date,
                'heartrate_category': row['heartrate_category'],
                'spo2_category': row['spo2_category'],
                'blood_pressure_systolic_category': row['blood_pressure_systolic_category'],
                'ankle_swelling': np.random.randint(1, 6),
                'breathlessness': np.random.randint(1, 6)
            })
            

    weekly_symptoms_df = pd.DataFrame(records)
    weekly_symptoms_df['average_symptoms'] = weekly_symptoms_df[['ankle_swelling', 'breathlessness']].mean(axis=1)
    weekly_symptoms_df['average_diff'] = weekly_symptoms_df.groupby('patient_id')['average_symptoms'].diff().fillna(0)
    
    # Determine the progress (worsening, improving, stable)
    def determine_progress(diff):
        if diff > 0:
            return 'Worsening'
        elif diff < 0:
            return 'Improving'
        else:
            return 'Stable'
    
    weekly_symptoms_df['progress'] = weekly_symptoms_df['average_diff'].apply(determine_progress)
    
    return weekly_symptoms_df

# Generate weekly symptoms data for combined_data
num_weeks = 52  # Adjust as needed
weekly_symptoms_combined_data = generate_weekly_symptoms_data(combined_data, num_weeks)

# Display the generated data
weekly_symptoms_combined_data.head()

patient_id                  timestamp  heartrate_category  spo2_category  \
0           1 2023-06-16 15:41:55.158612                   2              5   
1           1 2023-06-23 15:41:55.158612                   2              5   
2           1 2023-06-30 15:41:55.158612                   2              5   
3           1 2023-07-07 15:41:55.158612                   2              5   
4           1 2023-07-14 15:41:55.158612                   2              5   

   blood_pressure_systolic_category  ankle_swelling  breathlessness  \
0                                 2               2               2   
1                                 2               2               4   
2                                 2               1               4   
3                                 2               1               4   
4                                 2               1               3   

   average_symptoms  average_diff   progress  
0               2.0           0.0     Stable  
1               3.0           1.0  Worsening  
2               2.5          -0.5  Improving  
3               2.5           0.0     Stable  
4               2.0          -0.5  Improving

In [96]:
weekly_symptoms_combined_data['progress'].unique()

array(['Stable', 'Worsening', 'Improving'], dtype=object)

In [64]:

# Step 2: Get the most recent 'average_diff' and 'progress' for each patient
latest_weekly_data = weekly_symptoms_combined_data.sort_values(by='timestamp').groupby('patient_id').tail(1)[['patient_id', 'average_diff', 'progress']]

# Step 3: Merge the latest 'average_diff' and 'progress' with patient_data
combined_data1 = pd.merge(patient_data, latest_weekly_data, on='patient_id', how='left')

# Display the combined DataFrame with relevant columns

combined_data1.head()

patient_id  age  weight medical_conditions  heartrate  spo2  \
0           1   64      96               None         69    97   
1           2   67      66           Diabetes         90    95   
2           3   84      98             Anemia         76    97   
3           4   87      68             Anemia         99    95   
4           5   87      67               None         83    93   

   blood_pressure_systolic  ankle_swelling  breathlessness    drug  \
0                       91               1               5  Drug_C   
1                      120               2               5  Drug_B   
2                      108               1               4  Drug_A   
3                      116               3               2  Drug_C   
4                      125               5               3  Drug_D   

   average_diff   progress  
0           2.0  Worsening  
1          -3.5  Improving  
2           0.0     Stable  
3          -1.5  Improving  
4           1.5  Worsening

In [65]:
combined_data1.to_csv('combined_patient_data1.csv', index=False)

In [66]:
combined_data1.head()

patient_id  age  weight medical_conditions  heartrate  spo2  \
0           1   64      96               None         69    97   
1           2   67      66           Diabetes         90    95   
2           3   84      98             Anemia         76    97   
3           4   87      68             Anemia         99    95   
4           5   87      67               None         83    93   

   blood_pressure_systolic  ankle_swelling  breathlessness    drug  \
0                       91               1               5  Drug_C   
1                      120               2               5  Drug_B   
2                      108               1               4  Drug_A   
3                      116               3               2  Drug_C   
4                      125               5               3  Drug_D   

   average_diff   progress  
0           2.0  Worsening  
1          -3.5  Improving  
2           0.0     Stable  
3          -1.5  Improving  
4           1.5  Worsening

In [67]:
combined_data1['progress'].unique()

array(['Worsening', 'Improving', 'Stable'], dtype=object)

In [68]:
# Save the combined_data DataFrame to a CSV file
combined_data.to_csv('combined_patient_data.csv', index=False)


In [69]:
combined_data.shape

(5000, 15)

In [70]:
# Define a mapping for progress categories
progress_mapping = {'Worsening': 0, 'Improving': 1, 'Stable': 2}

# Convert 'progress' column to categorical values
combined_data1['progress_category'] = combined_data1['progress'].map(progress_mapping)
# Display the updated DataFrame
combined_data1.head()

patient_id  age  weight medical_conditions  heartrate  spo2  \
0           1   64      96               None         69    97   
1           2   67      66           Diabetes         90    95   
2           3   84      98             Anemia         76    97   
3           4   87      68             Anemia         99    95   
4           5   87      67               None         83    93   

   blood_pressure_systolic  ankle_swelling  breathlessness    drug  \
0                       91               1               5  Drug_C   
1                      120               2               5  Drug_B   
2                      108               1               4  Drug_A   
3                      116               3               2  Drug_C   
4                      125               5               3  Drug_D   

   average_diff   progress  progress_category  
0           2.0  Worsening                  0  
1          -3.5  Improving                  1  
2           0.0     Stable                  2  
3          -1.5  Improving                  1  
4           1.5  Worsening                  0

In [72]:
cat_cols = [col for col in combined_data1.columns if combined_data1[col].dtype=='object']

In [73]:
# Check the number of categories in the category column
for col in cat_cols:
    print(f"{col} has {combined_data1[col].nunique()} categories\n")

medical_conditions has 4 categories

drug has 4 categories

progress has 3 categories



In [75]:
num_cols = [col for col in combined_data1.columns if combined_data1[col].dtype!='object']

In [76]:
# Check the number of categories in the category column
for col in num_cols:
    print(f"{col} has {combined_data1[col].nunique()} categories\n")

patient_id has 5000 categories

age has 71 categories

weight has 71 categories

heartrate has 41 categories

spo2 has 11 categories

blood_pressure_systolic has 51 categories

ankle_swelling has 5 categories

breathlessness has 5 categories

average_diff has 17 categories

progress_category has 3 categories



In [84]:
combined_data1.columns

Index(['patient_id', 'age', 'weight', 'medical_conditions', 'heartrate',
       'spo2', 'blood_pressure_systolic', 'ankle_swelling', 'breathlessness',
       'drug', 'average_diff', 'progress', 'progress_category'],
      dtype='object')

In [77]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for col in cat_cols:
    combined_data1[col] = le.fit_transform(combined_data1[col])

In [122]:
combined_data1.head()

patient_id  age  weight  medical_conditions  heartrate  spo2  \
0           1   64      96                   3         69    97   
1           2   67      66                   1         90    95   
2           3   84      98                   0         76    97   
3           4   87      68                   0         99    95   
4           5   87      67                   3         83    93   

   blood_pressure_systolic  ankle_swelling  breathlessness  drug  \
0                       91               1               5     2   
1                      120               2               5     1   
2                      108               1               4     0   
3                      116               3               2     2   
4                      125               5               3     3   

   average_diff  progress  progress_category predicted_progress  
0           2.0         2                  0          Improving  
1          -3.5         0                  1          Worsening  
2           0.0         1                  2            Serious  
3          -1.5         0                  1          Worsening  
4           1.5         2                  0          Improving

In [146]:
# List of columns to drop
drop_cols = ['progress_category', 'patient_id', 'predicted_progress','average_diff','progress']

# Create a list of columns to keep
ind_col = [col for col in combined_data1.columns if col not in drop_cols]

# Create a new DataFrame with the remaining columns
new_data = combined_data1[ind_col]
new_data.head()

age  weight  medical_conditions  heartrate  spo2  blood_pressure_systolic  \
0   64      96                   3         69    97                       91   
1   67      66                   1         90    95                      120   
2   84      98                   0         76    97                      108   
3   87      68                   0         99    95                      116   
4   87      67                   3         83    93                      125   

   ankle_swelling  breathlessness  drug  
0               1               5     2  
1               2               5     1  
2               1               4     0  
3               3               2     2  
4               5               3     3

In [148]:
X = new_data
y = combined_data1['progress']

In [149]:
# lets create Training and Testing Sets for Validation of Results
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

print("The Shape of x train:", x_train.shape)
print("The Shape of x test:", x_test.shape)
print("The Shape of y train:", y_train.shape)
print("The Shape of y test:", y_test.shape)

The Shape of x train: (4000, 9)
The Shape of x test: (1000, 9)
The Shape of y train: (4000,)
The Shape of y test: (1000,)


In [150]:
# lets create a Predictive Model

from sklearn.linear_model import LogisticRegression

model1 = LogisticRegression()
model1.fit(x_train, y_train)
y_pred = model1.predict(x_test)

c:\Users\manid\anaconda3\envs\coding\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [151]:
X.head()

age  weight  medical_conditions  heartrate  spo2  blood_pressure_systolic  \
0   64      96                   3         69    97                       91   
1   67      66                   1         90    95                      120   
2   84      98                   0         76    97                      108   
3   87      68                   0         99    95                      116   
4   87      67                   3         83    93                      125   

   ankle_swelling  breathlessness  drug  
0               1               5     2  
1               2               5     1  
2               1               4     0  
3               3               2     2  
4               5               3     3

In [171]:
import numpy as np

# Function to get user input for the features
def get_input():
    age = int(input("Enter age: "))
    weight = int(input("Enter weight: "))
    medical_conditions = int(input("Enter number of medical conditions: "))
    heartrate = int(input("Enter heartrate: "))
    spo2 = int(input("Enter spo2: "))
    blood_pressure_systolic = int(input("Enter blood pressure systolic: "))
    ankle_swelling = int(input("Enter ankle swelling (0 for none, 1 for mild, 2 for moderate, 3 for severe): "))
    breathlessness = int(input("Enter breathlessness (0 for none, 1 for mild, 2 for moderate, 3 for severe): "))
    drug = int(input("Enter drug (0 for no drug, 1 for drug A, 2 for drug B, 3 for drug C): "))

    
    return np.array([[age, weight, medical_conditions, heartrate, spo2, blood_pressure_systolic, ankle_swelling, breathlessness, drug]])

# Get user input
input_features = get_input()

# Predict the target 'progress'
predicted_progress = y_pred.predict(input_features)

# Print the predicted progress category with advice
if predicted_progress == 2:
    print("Predicted progress: Serious")
    print("Your condition requires immediate medical attention. Please consult your doctor promptly.")
elif predicted_progress == 1:
    print("Predicted progress: Worsening")
    print("If your condition is worsening, consider increasing your frusemide dose. Monitor your condition more frequently and watch out for any new symptoms. Additionally, reduce your protein intake and fluid intake by 300 ml to avoid volume overload.")
elif predicted_progress == 0:
    print("Predicted progress: Improving")
    print("If your condition is improving, continue taking your medicines on time and monitor your condition regularly.")
else:
    print("Invalid progress prediction")


AttributeError: 'numpy.ndarray' object has no attribute 'predict'

In [ ]:

model.predict((np.array([[90,40,40,20,80,7,200]])))

In [90]:
X.shape

(5000, 12)

In [164]:
from sklearn.model_selection import train_test_split

# Divide the data set into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

In [165]:
from sklearn.preprocessing import StandardScaler

# Features normalization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [166]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Create KNN classifier
knn = KNeighborsClassifier()
# Train KNN classifier
knn.fit(X_train, y_train)
# Use the trained model for prediction
y_pred = knn.predict(X_test)

print(f'Training Accuracy of KNN is {accuracy_score(y_train, knn.predict(X_train))}\n')

# Calculate evaluation indicators
knn_accuracy = accuracy_score(y_test, y_pred)
knn_precision = precision_score(y_test, y_pred, average='macro')
knn_recall = recall_score(y_test, y_pred, average='macro')
knn_f1_score = f1_score(y_test, y_pred, average='macro')

print(f"Confusion Matrix: \n{confusion_matrix(y_test, y_pred)}\n")
print(f"Test Accuracy of KNN is {knn_accuracy} \n")
print(f"Test Precision of KNN is {knn_precision} \n")
print(f"Test Recall of KNN is {knn_recall} \n")
print(f"Test F1_score of KNN is {knn_f1_score} \n")
print(f"Classification Report: \n {classification_report(y_test, y_pred)}")


Training Accuracy of KNN is 0.6256

Confusion Matrix: 
[[663  47 390]
 [190  19 125]
 [579  41 446]]

Test Accuracy of KNN is 0.4512 

Test Precision of KNN is 0.36821960540510457 

Test Recall of KNN is 0.35933333060980205 

Test F1_score of KNN is 0.3499745612357712 

Classification Report: 
               precision    recall  f1-score   support

           0       0.46      0.60      0.52      1100
           1       0.18      0.06      0.09       334
           2       0.46      0.42      0.44      1066

    accuracy                           0.45      2500
   macro avg       0.37      0.36      0.35      2500
weighted avg       0.43      0.45      0.43      2500



In [169]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)

print(f'Training Accuracy of Gaussian Naive Bayes is {accuracy_score(y_train, gnb.predict(X_train))}\n')

gnb_accuracy = accuracy_score(y_test, y_pred)
gnb_precision = precision_score(y_test, y_pred, average='macro')
gnb_recall = recall_score(y_test, y_pred, average='macro')
gnb_f1_score = f1_score(y_test, y_pred, average='macro')

print(f"Confusion Matrix: \n{confusion_matrix(y_test, y_pred)}\n")
print(f"Test Accuracy of Gaussian Naive Bayes is {gnb_accuracy} \n")
print(f"Test Precision of Gaussian Naive Bayes is {gnb_precision} \n")
print(f"Test Recall of Gaussian Naive Bayes is {gnb_recall} \n")
print(f"Test F1_score of Gaussian Naive Bayes is {gnb_f1_score} \n")
print(f"Classification Report: \n{classification_report(y_test, y_pred)}")

Training Accuracy of Gaussian Naive Bayes is 0.4572

Confusion Matrix: 
[[627   1 472]
 [188   0 146]
 [629   0 437]]

Test Accuracy of Gaussian Naive Bayes is 0.4256 

Test Precision of Gaussian Naive Bayes is 0.2828095119314875 

Test Recall of Gaussian Naive Bayes is 0.3266479049405879 

Test F1_score of Gaussian Naive Bayes is 0.3016647689027663 

Classification Report: 
              precision    recall  f1-score   support

           0       0.43      0.57      0.49      1100
           1       0.00      0.00      0.00       334
           2       0.41      0.41      0.41      1066

    accuracy                           0.43      2500
   macro avg       0.28      0.33      0.30      2500
weighted avg       0.37      0.43      0.39      2500



In [170]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

print(f'Training Accuracy of Logistic Regression is {accuracy_score(y_train, logreg.predict(X_train))}\n')

logreg_accuracy = accuracy_score(y_test, y_pred)
logreg_precision = precision_score(y_test, y_pred, average='macro')
logreg_recall = recall_score(y_test, y_pred, average='macro')
logreg_f1_score = f1_score(y_test, y_pred, average='macro')

print(f"Confusion Matrix: \n{confusion_matrix(y_test, y_pred)}\n")
print(f"Test Accuracy of Logistic Regression is {logreg_accuracy} \n")
print(f"Test Precision of Logistic Regression is {logreg_precision} \n")
print(f"Test Recall of Logistic Regression is {logreg_recall} \n")
print(f"Test F1_score of Logistic Regression is {logreg_f1_score} \n")
print(f"Classification Report: \n{classification_report(y_test, y_pred)}")

Training Accuracy of Logistic Regression is 0.4596

Confusion Matrix: 
[[627   0 473]
 [194   0 140]
 [637   0 429]]

Test Accuracy of Logistic Regression is 0.4224 

Test Precision of Logistic Regression is 0.2805831352074332 

Test Recall of Logistic Regression is 0.32414634146341464 

Test F1_score of Logistic Regression is 0.2990825375018731 

Classification Report: 
              precision    recall  f1-score   support

           0       0.43      0.57      0.49      1100
           1       0.00      0.00      0.00       334
           2       0.41      0.40      0.41      1066

    accuracy                           0.42      2500
   macro avg       0.28      0.32      0.30      2500
weighted avg       0.36      0.42      0.39      2500



c:\Users\manid\anaconda3\envs\coding\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\manid\anaconda3\envs\coding\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\manid\anaconda3\envs\coding\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\U

In [98]:
logreg

LogisticRegression()

In [87]:
combined_data1.columns

Index(['patient_id', 'age', 'weight', 'medical_conditions', 'heartrate',
       'spo2', 'blood_pressure_systolic', 'ankle_swelling', 'breathlessness',
       'drug', 'average_diff', 'progress', 'progress_category'],
      dtype='object')

In [110]:
X_pred.head()

patient_id  age  weight  medical_conditions  heartrate  spo2  \
0           1   64      96                   3         69    97   
1           2   67      66                   1         90    95   
2           3   84      98                   0         76    97   
3           4   87      68                   0         99    95   
4           5   87      67                   3         83    93   

   blood_pressure_systolic  ankle_swelling  breathlessness  drug  \
0                       91               1               5     2   
1                      120               2               5     1   
2                      108               1               4     0   
3                      116               3               2     2   
4                      125               5               3     3   

   average_diff  progress  
0           2.0         2  
1          -3.5         0  
2           0.0         1  
3          -1.5         0  
4           1.5         2

In [ ]:
# Assume X_new is your new input data in the same format as X_train and X_test

# Preprocess X_new if needed

# Use the trained model to predict the target values
y_new_pred = logreg.predict(X_new)

# Print the predicted target values
print("Predicted target values:")
for y_pred in y_new_pred:
    if y_pred == 0:
        print("Improving")
    elif y_pred == 1:
        print("Stable")
    else:
        print("Worsening")

In [ ]:
from sklearn.linear_model import LogisticRegression

model1 = LogisticRegression()
model1.fit(x_train, y_train)
y_pred = model1.predict(x_test)

In [91]:
# Prepare the input data for predictions
X_pred = combined_data1[ind_col]

# Standardize the input data
X_pred_scaled = scaler.transform(X_pred)

# Make predictions
y_pred = logreg.predict(X_pred_scaled)

# Map predictions to progress categories
progress_mapping = {0: 'Improving', 1: 'Worsening', 2: 'Serious'}
predictions = [progress_mapping[pred] for pred in y_pred]

# Add predictions to combined_data1
combined_data1['predicted_progress'] = predictions

# Display the updated DataFrame with predictions
print(combined_data1[['patient_id', 'predicted_progress']].head())

   patient_id predicted_progress
0           1          Improving
1           2          Worsening
2           3            Serious
3           4          Worsening
4           5          Improving


In [ ]:
# Make predictions on the new data
predictions = model.predict(new_data)